In [31]:
!pip install kfp==1.8.9
!pip install google-cloud-pipeline-components==0.2.0

In [1]:
import kfp
from kfp import components
import kfp.dsl as dsl
from kfp.v2 import compiler
from kfp.v2.google.client import AIPlatformClient
from google_cloud_pipeline_components import aiplatform as gcc_aip

from components.preprocessing import split_data
from components.training import xgb_training
from pipelines.pipeline import xgb_pipeline

In [2]:
compiler.Compiler().compile(
    pipeline_func=xgb_pipeline,
    package_path="pl.json"
)


/opt/conda/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1266: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


ValueError: Invalid pipeline name: Xgb pipeline.
Please specify a pipeline name that matches the regular expression "^[a-z0-9][a-z0-9-]{0,127}$" using `dsl.pipeline(name=...)` decorator.

In [13]:
from google.cloud.aiplatform.pipeline_jobs import PipelineJob

pl = PipelineJob(display_name= 'xgb-job',
        template_path= "pl.json",
        location='us-central1',
        pipeline_root="gs://feature-store-mars21/vertex-xgb-ml-ops/a3b8252",
        parameter_values={'project_id': 'feature-store-mars21', 
                          'data_path': 'gs://mortgage_dataset_files/mortgage-small.csv'})

pl.run(sync=False)

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob


In [45]:

from google.cloud import aiplatform as aiplatform
df = aiplatform.get_pipeline_df(pipeline="xgb-pipeline")

In [46]:
df

,pipeline_name,run_name,param.input:endpoint_name,param.input:data_path,param.input:project_id,metric.auc,metric.confusionMatrix
0,xgb-pipeline,build-eb3b19f,xgb-ep,gs://mortgage_dataset_files/mortgage-small.csv,feature-store-mars21,0.970745,"{'rows': [{'row': [28.0, 0.0]}, {'row': [0.0, ..."
1,xgb-pipeline,xgb-pipeline-20211202194851,xgb-ep,gs://mortgage_dataset_files/mortgage-small.csv,feature-store-mars21,0.982732,"{'rows': [{'row': [26.0, 0.0]}, {'row': [0.0, ..."
2,xgb-pipeline,schedule-run-a3b8252-20211126t120000625z,xgb-ep,gs://mortgage_dataset_files/mortgage-small.csv,feature-store-mars21,0.982732,"{'rows': [{'row': [26.0, 0.0]}, {'row': [0.0, ..."
3,xgb-pipeline,schedule-run-a3b8252-20211126t090000554z,xgb-ep,gs://mortgage_dataset_files/mortgage-small.csv,feature-store-mars21,0.982732,"{'rows': [{'row': [26.0, 0.0]}, {'row': [0.0, ..."
4,xgb-pipeline,schedule-run-a3b8252-20211126t060000494z,xgb-ep,gs://mortgage_dataset_files/mortgage-small.csv,feature-store-mars21,0.982732,"{'annotationSpecs': [{'displayName': '0'}, {'d..."
...,...,...,...,...,...,...,...
105,xgb-pipeline,my-test-r7,xgb-ep,gs://mortgage_dataset_files/mortgage-small.csv,feature-store-mars21,0.975667,"{'rows': [{'row': [26.0, 0.0]}, {'row': [0.0, ..."
106,xgb-pipeline,my-test-r6,xgb-ep,gs://mortgage_dataset_files/mortgage-small.csv,feature-store-mars21,0.975667,"{'annotationSpecs': [{'displayName': '0'}, {'d..."
107,xgb-pipeline,my-test-r5,xgb-ep,gs://mortgage_dataset_files/mortgage-small.csv,feature-store-mars21,0.975667,"{'annotationSpecs': [{'displayName': '0'}, {'d..."
108,xgb-pipeline,my-test-r4,xgb-ep,gs://mortgage_dataset_files/mortgage-small.csv,feature-store-mars21,0.975667,"{'annotationSpecs': [{'displayName': '0'}, {'d..."


In [74]:
from google.cloud import aiplatform_v1 

PROJECT_ID='feature-store-mars21'
REGION='us-central1'

In [75]:
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
metadata_client = aiplatform_v1.MetadataServiceClient(
  client_options={
      "api_endpoint": API_ENDPOINT
  }
)

In [101]:
MODEL_FILTER="schema_title = \"system.Model\""
MODEL_FILTER="uri = \"gs://feature-store-mars21/vertex-xgb-ml-ops/27db53d/*\" AND schema_title=\"system.Model\""
artifact_request = aiplatform_v1.ListArtifactsRequest(
    parent="projects/{0}/locations/{1}/metadataStores/default".format(PROJECT_ID, REGION),
    filter=MODEL_FILTER
)
model_artifacts = metadata_client.list_artifacts(artifact_request)
model_artifacts

ListArtifactsPager<artifacts {
  name: "projects/202835066335/locations/us-central1/metadataStores/default/artifacts/13136319706365087704"
  display_name: "model"
  uri: "gs://feature-store-mars21/vertex-xgb-ml-ops/27db53d/202835066335/schedule-run-27db53d-20211202t220954595z-2/xgb-training_1071245382849134592/model"
  etag: "1638484767478"
  create_time {
    seconds: 1638484664
    nanos: 914000000
  }
  update_time {
    seconds: 1638484767
    nanos: 478000000
  }
  state: LIVE
  schema_title: "system.Model"
  schema_version: "0.0.1"
  metadata {
  }
}
artifacts {
  name: "projects/202835066335/locations/us-central1/metadataStores/default/artifacts/4246735858793993241"
  display_name: "model"
  uri: "gs://feature-store-mars21/vertex-xgb-ml-ops/27db53d/202835066335/schedule-run-27db53d-20211202t220954595z-1/xgb-training_-6224586013491068928/model"
  etag: "1638484219238"
  create_time {
    seconds: 1638484035
    nanos: 599000000
  }
  update_time {
    seconds: 1638484219
    nano

In [117]:
PL_FILTER=""
context_request = aiplatform_v1.ListContextsRequest(
    parent="projects/{0}/locations/{1}/metadataStores/default".format(PROJECT_ID, REGION),
    filter=PL_FILTER)

model_artifacts = metadata_client.list_contexts(context_request)
model_artifacts

ListContextsPager<contexts {
  name: "projects/202835066335/locations/us-central1/metadataStores/default/contexts/schedule-run-27db53d-20211202t220954595z-2"
  display_name: "schedule-run-27db53d-20211202t220954595z-2"
  etag: "1638484529892"
  create_time {
    seconds: 1638484529
    nanos: 892000000
  }
  update_time {
    seconds: 1638484529
    nanos: 892000000
  }
  parent_contexts: "projects/202835066335/locations/us-central1/metadataStores/default/contexts/xgb-pipeline"
  schema_title: "system.PipelineRun"
  schema_version: "0.0.1"
  metadata {
  }
}
contexts {
  name: "projects/202835066335/locations/us-central1/metadataStores/default/contexts/schedule-run-27db53d-20211202t220954595z-1"
  display_name: "schedule-run-27db53d-20211202t220954595z-1"
  etag: "1638483839184"
  create_time {
    seconds: 1638483839
    nanos: 184000000
  }
  update_time {
    seconds: 1638483839
    nanos: 184000000
  }
  parent_contexts: "projects/202835066335/locations/us-central1/metadataStores/d